[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/docs/use_cases/web_scraping.ipynb)

## Use case

[Web research](https://blog.langchain.dev/automating-web-research/) is one of the killer LLM applications:

* Users have [highlighted it](https://twitter.com/GregKamradt/status/1679913813297225729?s=20) as one of his top desired AI tools.
* OSS repos like [gpt-researcher](https://github.com/assafelovic/gpt-researcher) are growing in popularity.

![Image description](https://github.com/langchain-ai/langchain/blob/master/docs/static/img/web_scraping.png?raw=1)

## Overview

Gathering content from the web has a few components:

* `Search`: Query to url (e.g., using `GoogleSearchAPIWrapper`).
* `Loading`: Url to HTML  (e.g., using `AsyncHtmlLoader`, `AsyncChromiumLoader`, etc).
* `Transforming`: HTML to formatted text (e.g., using `HTML2Text` or `Beautiful Soup`).

## Quickstart

In [10]:
#pip install -q langchain-openai langchain playwright beautifulsou!4
!pip install playwright --upgrade

# Set env var OPENAI_API_KEY or load from a .env file:
# import dotenv
# dotenv.load_dotenv()

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 99.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.0/620.0 kB 86.0 MB/s eta 0:00:00
  Attempting uninstall: greenlet
    Found existing installation: greenlet 3.0.0
    Uninstalling greenlet-3.0.0:
      Successfully uninstalled greenlet-3.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-legacy 0.9.48 requires networkx>=3.0, but you have networkx 2.6.3 which is incompatible.
llama-index-core 0.10.15 requires networkx>=3.0, but you have networkx 2.6.3 which is incompatible.


Scraping HTML content using a headless instance of Chromium.

* The async nature of the scraping process is handled using Python's asyncio library.
* The actual interaction with the web pages is handled by Playwright.

In [1]:
from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_transformers import BeautifulSoupTransformer

# Load HTML
loader = AsyncChromiumLoader(["https://www.wsj.com"])
html = loader.load()

RuntimeError: asyncio.run() cannot be called from a running event loop

Scrape text content tags such as `<p>, <li>, <div>, and <a>` tags from the HTML content:

* `<p>`: The paragraph tag. It defines a paragraph in HTML and is used to group together related sentences and/or phrases.

* `<li>`: The list item tag. It is used within ordered (`<ol>`) and unordered (`<ul>`) lists to define individual items within the list.

* `<div>`: The division tag. It is a block-level element used to group other inline or block-level elements.

* `<a>`: The anchor tag. It is used to define hyperlinks.

* `<span>`:  an inline container used to mark up a part of a text, or a part of a document.

For many news websites (e.g., WSJ, CNN), headlines and summaries are all in `<span>` tags.

In [ ]:
# Transform
bs_transformer = BeautifulSoupTransformer()
docs_transformed = bs_transformer.transform_documents(html, tags_to_extract=["span"])

In [ ]:
# Result
docs_transformed[0].page_content[0:500]

'English EditionEnglish中文 (Chinese)日本語 (Japanese) More Other Products from WSJBuy Side from WSJWSJ ShopWSJ Wine Other Products from WSJ Search Quotes and Companies Search Quotes and Companies 0.15% 0.03% 0.12% -0.42% 4.102% -0.69% -0.25% -0.15% -1.82% 0.24% 0.19% -1.10% About Evan His Family Reflects His Reporting How You Can Help Write a Message Life in Detention Latest News Get Email Updates Four Americans Released From Iranian Prison The Americans will remain under house arrest until they are '

These `Documents` now are staged for downstream usage in various LLM apps, as discussed below.

## Loader

### AsyncHtmlLoader

The [AsyncHtmlLoader](/docs/integrations/document_loaders/async_html) uses the `aiohttp` library to make asynchronous HTTP requests, suitable for simpler and lightweight scraping.

### AsyncChromiumLoader

The [AsyncChromiumLoader](/docs/integrations/document_loaders/async_chromium) uses Playwright to launch a Chromium instance, which can handle JavaScript rendering and more complex web interactions.

Chromium is one of the browsers supported by Playwright, a library used to control browser automation.

Headless mode means that the browser is running without a graphical user interface, which is commonly used for web scraping.

In [2]:
from langchain_community.document_loaders import AsyncHtmlLoader

urls = ["https://www.espn.com", "https://lilianweng.github.io/posts/2023-06-23-agent/"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()

Fetching pages: 100%|#######################################################################################################################################| 2/2 [00:00<00:00, 16.33it/s]


## Transformer

### HTML2Text

[HTML2Text](/docs/integrations/document_transformers/html2text) provides a straightforward conversion of HTML content into plain text (with markdown-like formatting) without any specific tag manipulation.

It's best suited for scenarios where the goal is to extract human-readable text without needing to manipulate specific HTML elements.

### Beautiful Soup

Beautiful Soup offers more fine-grained control over HTML content, enabling specific tag extraction, removal, and content cleaning.

It's suited for cases where you want to extract specific information and clean up the HTML content according to your needs.

In [ ]:
#!pip install hml2text

In [2]:
from langchain_community.document_loaders import AsyncHtmlLoader

urls = ["https://www.espn.com", "https://lilianweng.github.io/posts/2023-06-23-agent/"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()

Fetching pages: 100%|######################################################################################################################################| 2/2 [00:00<00:00, 37.05it/s]


In [4]:
from langchain_community.document_transformers import Html2TextTransformer

html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)
docs_transformed[0].page_content[0:500]

'Skip to main content  Skip to navigation\n\n<\n\n>\n\nMenu\n\n## ESPN\n\n  *   *   * scores\n\n  * NFL\n  * NBA\n  * MLB\n  * NHL\n  * Soccer\n  * MMA\n  * …\n\n    * NCAAF\n    * Sports Betting\n    * Boxing\n    * CFL\n    * NCAA\n    * Cricket\n    * F1\n    * Golf\n    * Horse\n    * LLWS\n    * NASCAR\n    * NBA G League\n    * NCAAM\n    * NCAAW\n    * Olympic Sports\n    * PLL\n    * Professional Wrestling\n    * Racing\n    * RN BB\n    * RN FB\n    * Rugby\n    * Tennis\n    * WNBA\n    * X Games\n    * UFL\n\n  * More ESPN\n  * Fan'

## Scraping with extraction

### LLM with function calling

Web scraping is challenging for many reasons.

One of them is the changing nature of modern websites' layouts and content, which requires modifying scraping scripts to accommodate the changes.

Using Function (e.g., OpenAI) with an extraction chain, we avoid having to change your code constantly when websites change.

We're using `gpt-3.5-turbo-0613` to guarantee access to OpenAI Functions feature (although this might be available to everyone by time of writing).

We're also keeping `temperature` at `0` to keep randomness of the LLM down.

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

### Define a schema

Next, you define a schema to specify what kind of data you want to extract.

Here, the key names matter as they tell the LLM what kind of information they want.

So, be as detailed as possible.

In this example, we want to scrape only news article's name and summary from The Wall Street Journal website.

In [2]:
from langchain.chains import create_extraction_chain

schema = {
    "properties": {
        "news_article_title": {"type": "string"},
        "news_article_summary": {"type": "string"},
    },
    "required": ["news_article_title", "news_article_summary"],
}


def extract(content: str, schema: dict):
    return create_extraction_chain(schema=schema, llm=llm).run(content)

### Run the web scraper w/ BeautifulSoup

As shown above, we'll be using `BeautifulSoupTransformer`.

In [6]:
import pprint
#import nest_asyncio

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.chromium import AsyncChromiumLoader

def scrape_with_playwright(urls, schema):
    loader = AsyncChromiumLoader(urls)
    #nest_asyncio.apply()
    docs = loader.load()
    bs_transformer = BeautifulSoupTransformer()
    docs_transformed = bs_transformer.transform_documents(
        docs, tags_to_extract=["span"]
    )
    print("Extracting content with LLM")

    # Grab the first 1000 tokens of the site
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=1000, chunk_overlap=0
    )
    splits = splitter.split_documents(docs_transformed)

    # Process the first split
    extracted_content = extract(schema=schema, content=splits[0].page_content)
    pprint.pprint(extracted_content)
    return extracted_content


urls = ["https://www.wsj.com"]
extracted_content = scrape_with_playwright(urls, schema=schema)

Exception: Connection closed while reading from the driver

We can compare the headlines scraped to the page:

![Image description](https://github.com/langchain-ai/langchain/blob/master/docs/static/img/wsj_page.png?raw=1)

Looking at the [LangSmith trace](https://smith.langchain.com/public/c3070198-5b13-419b-87bf-3821cdf34fa6/r), we can see what is going on under the hood:

* It's following what is explained in the [extraction](docs/use_cases/extraction).
* We call the `information_extraction` function on the input text.
* It will attempt to populate the provided schema from the url content.

## Research automation

Related to scraping, we may want to answer specific questions using searched content.

We can automate the process of [web research](https://blog.langchain.dev/automating-web-research/) using a retriever, such as the `WebResearchRetriever`.

![Image description](https://github.com/langchain-ai/langchain/blob/master/docs/static/img/web_research.png?raw=1)

Copy requirements [from here](https://github.com/langchain-ai/web-explorer/blob/main/requirements.txt):

`pip install -r requirements.txt`

Set `GOOGLE_CSE_ID` and `GOOGLE_API_KEY`.

In [25]:
from langchain.retrievers.web_research import WebResearchRetriever
from langchain_chroma import Chroma
from langchain_community.utilities import GoogleSearchAPIWrapper
#from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.chat_models import BedrockChat
import boto3
import os

In [26]:
# Vectorstore
vectorstore = Chroma(
    #embedding_function=OpenAIEmbeddings(), persist_directory="./chroma_db_oai"
    embedding_function=OpenAIEmbeddings()
)

# LLM
bedrock_client = boto3.client("bedrock-runtime", region_name='us-west-2')
model_kwargs =  { 
            "max_tokens": 2048,
            "temperature": 0.1,
            "top_k": 40,
            "top_p": 0.90,
            #"stop_sequences":'\n\n\Human',
        }
    
llm = BedrockChat(model_id='anthropic.claude-3-haiku-20240307-v1:0',client=bedrock_client, model_kwargs=model_kwargs)
#llm = ChatOpenAI(temperature=0)

# Search
search = GoogleSearchAPIWrapper(google_api_key=os.getenv("google_api_key"), google_cse_id=os.getenv("google_cse_id"))

Initialize retriever with the above tools to:
    
* Use an LLM to generate multiple relevant search queries (one LLM call)
* Execute a search for each query
* Choose the top K links per query  (multiple search calls in parallel)
* Load the information from all chosen links (scrape pages in parallel)
* Index those documents into a vectorstore
* Find the most relevant documents for each original generated search query

In [27]:
# Initialize
web_research_retriever = WebResearchRetriever.from_llm(
    vectorstore=vectorstore, llm=llm, search=search
)

In [42]:
%%time

# Retrival chain
from langchain.prompts.chat import ChatPromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

messages = [
    ("system", """Your are a helpful assistant to provide omprehensive and truthful answers to questions, \n
                drawing upon all relevant information contained within the specified in {context}. \n 
                You add value by analyzing the situation and offering insights to enrich your answer. \n
                Simply say I don't know if you can not find any evidence to match the question. \n
                """),
    #MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
]
prompt_template = ChatPromptTemplate.from_messages(messages)

# Reranker
compression_retriever = ContextualCompressionRetriever(
    base_compressor= FlashrankRerank(), base_retriever=web_research_retriever
)

rag_chain = (
    #{"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
    #| RunnableParallel(answer=hub.pull("rlm/rag-prompt") | chat |format_docs, question=itemgetter("question") ) 
    RunnableParallel(context=compression_retriever | format_docs, question=RunnablePassthrough() )
    | prompt_template
    | llm
    | StrOutputParser()
)

user_input = "How do LLM Powered Autonomous Agents work?"
result = rag_chain.invoke(user_input)
result

INFO:langchain.retrievers.web_research:Generating questions for Google Search ...
INFO:langchain.retrievers.web_research:Questions for Google Search (raw): {'question': 'How do LLM Powered Autonomous Agents work?', 'text': ['1. What are the key components of LLM-powered autonomous agents?\n', '2. How do LLM-based systems enable autonomous decision-making?\n', '3. What are the underlying principles behind the functioning of LLM-powered autonomous agents?']}
INFO:langchain.retrievers.web_research:Questions for Google Search: ['1. What are the key components of LLM-powered autonomous agents?\n', '2. How do LLM-based systems enable autonomous decision-making?\n', '3. What are the underlying principles behind the functioning of LLM-powered autonomous agents?']
INFO:langchain.retrievers.web_research:Searching for relevant urls...
INFO:langchain.retrievers.web_research:Searching for relevant urls...
INFO:langchain.retrievers.web_research:Search results: [{'title': "LLM Powered Autonomous Agen

CPU times: user 1min 21s, sys: 178 ms, total: 1min 21s
Wall time: 11.9 s


'Here\'s an overview of how LLM-powered autonomous agents work:\n\n1. Agent System Overview:\n   - The agent uses a large language model (LLM) as the core controller or "brain".\n   - The LLM is complemented by several key components:\n     - Planning: Task decomposition, self-reflection to learn from past experiences.\n     - Memory: Storing and retrieving relevant information using techniques like maximum inner product search (MIPS).\n     - Tool Use: Interfacing with external tools, APIs, and environments to accomplish tasks.\n\n2. Component One: Planning\n   - Task Decomposition: The agent breaks down high-level goals into smaller, actionable steps.\n   - Self-Reflection: The agent reflects on its past actions and outcomes to learn and improve its planning abilities.\n\n3. Component Two: Memory\n   - Types of Memory: The agent utilizes different types of memory, such as episodic memory (experiences), semantic memory (facts), and procedural memory (skills).\n   - Maximum Inner Produ

In [45]:
%%time

# Run
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.web_research").setLevel(logging.INFO)
from langchain.chains import RetrievalQAWithSourcesChain

user_input = "How do LLM Powered Autonomous Agents work?"


qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm, retriever=web_research_retriever
)
result = qa_chain({"question": user_input})
result

INFO:langchain.retrievers.web_research:Generating questions for Google Search ...
INFO:langchain.retrievers.web_research:Questions for Google Search (raw): {'question': 'How do LLM Powered Autonomous Agents work?', 'text': ['1. What are the key components of LLM-powered autonomous agents?\n', '2. How do LLM-based systems enable autonomous decision-making?\n', '3. What are the underlying principles behind the functioning of LLM-powered autonomous agents?']}
INFO:langchain.retrievers.web_research:Questions for Google Search: ['1. What are the key components of LLM-powered autonomous agents?\n', '2. How do LLM-based systems enable autonomous decision-making?\n', '3. What are the underlying principles behind the functioning of LLM-powered autonomous agents?']
INFO:langchain.retrievers.web_research:Searching for relevant urls...
INFO:langchain.retrievers.web_research:Searching for relevant urls...
INFO:langchain.retrievers.web_research:Search results: [{'title': "LLM Powered Autonomous Agen

CPU times: user 37.3 ms, sys: 4.01 ms, total: 41.3 ms
Wall time: 6.26 s


{'question': 'How do LLM Powered Autonomous Agents work?',
 'answer': 'I do not have enough information to provide a detailed answer on how LLM-powered autonomous agents work. The provided content focuses on the general components and challenges of such systems, but does not go into the specifics of how they actually function. Without more detailed technical information, I cannot confidently describe the inner workings of these agents. I apologize that I cannot give a more complete answer to this question.\n\n',
 'sources': ''}

### Going deeper

* Here's a [app](https://github.com/langchain-ai/web-explorer/tree/main) that wraps this retriever with a lighweight UI.

## Question answering over a website

To answer questions over a specific website, you can use Apify's [Website Content Crawler](https://apify.com/apify/website-content-crawler) Actor, which can deeply crawl websites such as documentation, knowledge bases, help centers, or blogs,
and extract text content from the web pages.

In the example below, we will deeply crawl the Python documentation of LangChain's Chat LLM models and answer a question over it.

First, install the requirements
`pip install apify-client langchain-openai langchain`

Next, set `OPENAI_API_KEY` and `APIFY_API_TOKEN` in your environment variables.

The full code follows:

In [24]:
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.docstore.document import Document
from langchain_community.utilities import ApifyWrapper

apify = ApifyWrapper()
# Call the Actor to obtain text from the crawled webpages
loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
    run_input={"startUrls": [{"url": "/docs/integrations/chat/"}]},
    dataset_mapping_function=lambda item: Document(
        page_content=item["text"] or "", metadata={"source": item["url"]}
    ),
)

# Create a vector store based on the crawled data
index = VectorstoreIndexCreator().from_loaders([loader])

# Query the vector store
query = "Are any OpenAI chat models integrated in LangChain?"
result = index.query(query)
print(result)

ValidationError: 1 validation error for ApifyWrapper
__root__
  Did not find apify_api_token, please add an environment variable `APIFY_API_TOKEN` which contains it, or pass `apify_api_token` as a named parameter. (type=value_error)